In [ ]:
import tkinter as tk
from PIL import Image, ImageTk
import datetime
import sqlite3

# Initialize SQLite database
conn = sqlite3.connect("call_data.db")
cursor = conn.cursor()

# Create a table to store call data if it doesn't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS call_history (
        user_ip TEXT,
        call_start_time DATETIME,
        call_end_time DATETIME,
        call_duration INTEGER
    )
''')
conn.commit()

# Functions for the different options
def start_listening():
    target_ip = target_ip_entry.get()
    start_time = datetime.datetime.now()
    user_call_start_times[target_ip] = start_time

def end_call():
    target_ip = target_ip_entry.get()
    if target_ip in user_call_start_times:
        start_time = user_call_start_times[target_ip]
        end_time = datetime.datetime.now()
        call_duration = (end_time - start_time).total_seconds()

        # Store the call data in the database
        cursor.execute("INSERT INTO call_history (user_ip, call_start_time, call_end_time, call_duration) VALUES (?, ?, ?, ?)",
                       (target_ip, start_time, end_time, call_duration))
        conn.commit()

        print(f"User {target_ip} was in the call for {call_duration} seconds")
    else:
        print(f"User {target_ip} did not start a call.")

# Main window
root = tk.Tk()
root.title("Zoom Clone")

# Icon
icon_image = Image.open("zoom.png")
icon_photo = ImageTk.PhotoImage(icon_image)
root.iconphoto(True, icon_photo)

# Entry for Target IP
target_ip_label = tk.Label(root, text="Enter Target IP:")
target_ip_label.grid(row=0, column=0, padx=5, pady=5)
target_ip_entry = tk.Entry(root)
target_ip_entry.grid(row=0, column=1, padx=5, pady=5)

# Create a 2x2 grid for image buttons
button_width = 70
button_height = 70

listening_image = Image.open("play.png")
listening_image = listening_image.resize((button_width, button_height))
listening_photo = ImageTk.PhotoImage(listening_image)

end_call_image = Image.open("stop.png")
end_call_image = end_call_image.resize((button_width, button_height))
end_call_photo = ImageTk.PhotoImage(end_call_image)

listening_button = tk.Button(root, image=listening_photo, command=start_listening, width=button_width, height=button_height, compound=tk.TOP)
end_call_button = tk.Button(root, image=end_call_photo, command=end_call, width=button_width, height=button_height, compound=tk.TOP)

listening_button.grid(row=1, column=0, padx=5, pady=5)
end_call_button.grid(row=1, column=1, padx=5, pady=5)

root.mainloop()

# Close the database connection when the application exits
conn.close()

In [ ]:
import tkinter as tk
from PIL import Image, ImageTk
from vidstream import *
import socket
import threading
import time
import sqlite3

# Initialize dictionary to store call start times
call_start_times = {}

local_ip_address = socket.gethostbyname(socket.gethostname())
server = StreamingServer(local_ip_address, 9813)
reciever = AudioReceiver(local_ip_address, 8713)

# Initialize SQLite database
conn = sqlite3.connect("call_data.db")
cursor = conn.cursor()

# Create a table to store call data if it doesn't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS call_history (
        user_ip TEXT,
        call_start_time DATETIME,
        call_end_time DATETIME,
        call_duration INTEGER
    )
''')
conn.commit()

def start_listening():
    t1 = threading.Thread(target=server.start_server)
    t2 = threading.Thread(target=reciever.start_server)
    t1.start()
    t2.start()
    # Record the start time of the call
    call_start_times[local_ip_address] = time.time()

def end_call(ip):
    if ip in call_start_times:
        start_time = call_start_times[ip]
        end_time = time.time()
        call_duration = int(end_time - start_time)

        # Store the call data in the database
        cursor.execute("INSERT INTO call_history (user_ip, call_start_time, call_end_time, call_duration) VALUES (?, ?, ?, ?)",
                       (ip, start_time, end_time, call_duration))
        conn.commit()

        print(f"Call with {ip} lasted for {call_duration} seconds")
    else:
        print(f"No call started with {ip}.")

def start_camera_stream():
    target_ip = target_ip_entry.get()
    camera_client = CameraClient(target_ip, 7613)
    t3 = threading.Thread(target=camera_client.start_stream)
    t3.start()

def start_screen_sharing():
    target_ip = target_ip_entry.get()
    screen_client = ScreenShareClient(target_ip, 7613)
    t4 = threading.Thread(target=screen_client.start_stream)
    t4.start()
    
def start_audio_stream():
    target_ip = target_ip_entry.get()
    audio_sender = AudioSender(target_ip, 6713)
    t5 = threading.Thread(target=audio_sender.start_stream)
    t5.start()

# Main window
root = tk.Tk()
root.title("Zoom Clone")

# Icon
iconpath = r"C:\Users\KIIT\Desktop\5th sem materials\Project\zoom.ico"
root.iconbitmap(iconpath)

# Entry for Target IP
target_ip_label = tk.Label(root, text="Enter Target IP:")
target_ip_label.grid(row=0, column=0, padx=5, pady=5)
target_ip_entry = tk.Entry(root)
target_ip_entry.grid(row=0, column=1, padx=5, pady=5)

# Create a 2x2 grid for image buttons
button_width = 70
button_height = 70

listening_image = Image.open("play.png")
listening_image = listening_image.resize((button_width, button_height))
listening_photo = ImageTk.PhotoImage(listening_image)

end_call_image = Image.open("call.png")
end_call_image = end_call_image.resize((button_width, button_height))
end_call_photo = ImageTk.PhotoImage(end_call_image)

camera_image = Image.open("camera.png")
camera_image = camera_image.resize((button_width, button_height))
camera_photo = ImageTk.PhotoImage(camera_image)

screen_share_image = Image.open("screen.png")
screen_share_image = screen_share_image.resize((button_width, button_height))
screen_share_photo = ImageTk.PhotoImage(screen_share_image)

audio_stream_image = Image.open("mic.png")
audio_stream_image = audio_stream_image.resize((button_width, button_height))
audio_stream_photo = ImageTk.PhotoImage(audio_stream_image)

listening_button = tk.Button(root, image=listening_photo, command=start_listening, width=button_width, height=button_height, compound=tk.TOP)
end_call_button = tk.Button(root, image=end_call_photo, command=lambda: end_call(target_ip_entry.get()), width=button_width, height=button_height, compound=tk.TOP)
camera_stream_button = tk.Button(root, image=camera_photo, command=start_camera_stream, width=button_width, height=button_height, compound=tk.TOP)
screen_share_button = tk.Button(root, image=screen_share_photo, command=start_screen_sharing, width=button_width, height=button_height, compound=tk.TOP)
audio_stream_button = tk.Button(root, image=audio_stream_photo, command=start_audio_stream, width=button_width, height=button_height, compound=tk.TOP)

listening_button.grid(row=1, column=0, padx=5, pady=5)
end_call_button.grid(row=1, column=1, padx=5, pady=5)
camera_stream_button.grid(row=2, column=0, padx=5, pady=5)
screen_share_button.grid(row=2, column=1, padx=5, pady=5)
audio_stream_button.grid(row=3, column=0, padx=5, pady=5)

root.mainloop()

# Close the database connection when the application exits
conn.close()

In [3]:
import tkinter as tk
from PIL import Image, ImageTk
from vidstream import *
import socket
import threading
import requests

local_ip_address = socket.gethostbyname(socket.gethostname())
#print(local_ip_address)
server = StreamingServer(local_ip_address, 9553)
reciever = AudioReceiver(local_ip_address, 8553)

def start_listening():
    t1 = threading.Thread(target=server.start_server)
    t2 = threading.Thread(target=reciever.start_server)
    t1.start()
    t2.start()
    
    
def start_camera_stream():
    target_ip = target_ip_entry.get()
    camera_client = CameraClient(target_ip, 7553)
    t3 = threading.Thread(target=camera_client.start_stream)
    t3.start()

def start_screen_sharing():
    target_ip = target_ip_entry.get()
    screen_client = ScreenShareClient(target_ip, 7553)
    t4 = threading.Thread(target=screen_client.start_stream)
    t4.start()
    
def start_audio_stream():
    target_ip = target_ip_entry.get()
    audio_sender = AudioSender(target_ip, 6523)
    t5 = threading.Thread(target=audio_sender.start_stream)
    t5.start()



# Main window
root = tk.Tk()
root.title("Zoom Clone")

# Icon
#icon_image = Image.open("zoom.png")
#icon_photo = ImageTk.PhotoImage(icon_image)
#root.iconphoto(True, icon_photo)

# Icon
iconpath = r"C:\Users\KIIT\Desktop\5th sem materials\Project\zoom.ico"
root.iconbitmap(iconpath)

# Entry for Target IP
target_ip_label = tk.Label(root, text="Enter Target IP:")
target_ip_label.grid(row=0, column=0, padx=5, pady=5)
target_ip_entry = tk.Entry(root)
target_ip_entry.grid(row=0, column=1, padx=5, pady=5)

# Create a 2x2 grid for image buttons
button_width = 70
button_height = 70

listening_image = Image.open("play.png")
listening_image = listening_image.resize((button_width, button_height))
listening_photo = ImageTk.PhotoImage(listening_image)

camera_image = Image.open("camera.png")
camera_image = camera_image.resize((button_width, button_height))
camera_photo = ImageTk.PhotoImage(camera_image)

screen_share_image = Image.open("screen.png")
screen_share_image = screen_share_image.resize((button_width, button_height))
screen_share_photo = ImageTk.PhotoImage(screen_share_image)

audio_stream_image = Image.open("mic.png")
audio_stream_image = audio_stream_image.resize((button_width, button_height))
audio_stream_photo = ImageTk.PhotoImage(audio_stream_image)

listening_button = tk.Button(root, image=listening_photo, command=start_listening, width=button_width, height=button_height, compound=tk.TOP)
camera_stream_button = tk.Button(root, image=camera_photo, command=start_camera_stream, width=button_width, height=button_height, compound=tk.TOP)
screen_share_button = tk.Button(root, image=screen_share_photo, command=start_screen_sharing, width=button_width, height=button_height, compound=tk.TOP)
audio_stream_button = tk.Button(root, image=audio_stream_photo, command=start_audio_stream, width=button_width, height=button_height, compound=tk.TOP)

listening_button.grid(row=1, column=0, padx=5, pady=5)
camera_stream_button.grid(row=1, column=1, padx=5, pady=5)
screen_share_button.grid(row=2, column=0, padx=5, pady=5)
audio_stream_button.grid(row=2, column=1, padx=5, pady=5)

root.mainloop()

Exception in thread Thread-10 (__client_streaming):
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\anaconda3\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\KIIT\AppData\Roaming\Python\Python310\site-packages\vidstream\audio.py", line 52, in __client_streaming
    self.__sending_socket.connect((self.__host, self.__port))
TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond
Exception in thread Thread-12 (__client_streaming):
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\anaconda3\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  F

In [1]:
import socket

try:
    ip_address = socket.gethostbyname("example.com")
    print(f"Resolved IP address: {ip_address}")
except socket.gaierror as e:
    print(f"DNS resolution failed: {str(e)}")

Resolved IP address: 93.184.216.34


In [17]:
import tkinter as tk
from PIL import Image, ImageTk
from vidstream import *
import socket
import threading
import requests

local_ip_address = socket.gethostbyname(socket.gethostname())
server = StreamingServer(local_ip_address, 6593)
reciever = AudioReceiver(local_ip_address, 9583)

def start_listening():
    try:
        t1 = threading.Thread(target=server.start_server)
        t2 = threading.Thread(target=reciever.start_server)
        t1.start()
        t2.start()
    except Exception as e:
        print(f"Error starting listening threads: {e}")

def start_camera_stream():
    target_ip = target_ip_entry.get()
    camera_client = CameraClient(target_ip, 7162)
    try:
        t3 = threading.Thread(target=camera_client.start_stream)
        t3.start()
    except Exception as e:
        print(f"Error starting camera streaming: {e}")

def start_screen_sharing():
    target_ip = target_ip_entry.get()
    screen_client = ScreenShareClient(target_ip, 7162)
    try:
        t4 = threading.Thread(target=screen_client.start_stream)
        t4.start()
    except Exception as e:
        print(f"Error starting screen sharing: {e}")

def start_audio_stream():
    target_ip = target_ip_entry.get()
    audio_sender = AudioSender(target_ip, 8172)
    try:
        t5 = threading.Thread(target=audio_sender.start_stream)
        t5.start()
    except Exception as e:
        print(f"Error starting audio streaming: {e}")

# Main window
root = tk.Tk()
root.title("Zoom Clone")

# Icon
iconpath = r"C:\Users\KIIT\Desktop\5th sem materials\Project\zoom.ico"
root.iconbitmap(iconpath)

# Entry for Target IP
target_ip_label = tk.Label(root, text="Enter Target IP:")
target_ip_label.grid(row=0, column=0, padx=5, pady=5)
target_ip_entry = tk.Entry(root)
target_ip_entry.grid(row=0, column=1, padx=5, pady=5)

# Create a 2x2 grid for image buttons
button_width = 70
button_height = 70

listening_image = Image.open("play.png")
listening_image = listening_image.resize((button_width, button_height))
listening_photo = ImageTk.PhotoImage(listening_image)

camera_image = Image.open("camera.png")
camera_image = camera_image.resize((button_width, button_height))
camera_photo = ImageTk.PhotoImage(camera_image)

screen_share_image = Image.open("screen.png")
screen_share_image = screen_share_image.resize((button_width, button_height))
screen_share_photo = ImageTk.PhotoImage(screen_share_image)

audio_stream_image = Image.open("mic.png")
audio_stream_image = audio_stream_image.resize((button_width, button_height))
audio_stream_photo = ImageTk.PhotoImage(audio_stream_image)

listening_button = tk.Button(root, image=listening_photo, command=start_listening, width=button_width, height=button_height, compound=tk.TOP)
camera_stream_button = tk.Button(root, image=camera_photo, command=start_camera_stream, width=button_width, height=button_height, compound=tk.TOP)
screen_share_button = tk.Button(root, image=screen_share_photo, command=start_screen_sharing, width=button_width, height=button_height, compound=tk.TOP)
audio_stream_button = tk.Button(root, image=audio_stream_photo, command=start_audio_stream, width=button_width, height=button_height, compound=tk.TOP)

listening_button.grid(row=1, column=0, padx=5, pady=5)
camera_stream_button.grid(row=1, column=1, padx=5, pady=5)
screen_share_button.grid(row=2, column=0, padx=5, pady=5)
audio_stream_button.grid(row=2, column=1, padx=5, pady=5)

root.mainloop()

Exception in thread Thread-32 (__client_streaming):
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\anaconda3\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\KIIT\AppData\Roaming\Python\Python310\site-packages\vidstream\streaming.py", line 269, in __client_streaming
    self.__client_socket.connect((self.__host, self.__port))
TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond
Exception in thread Thread-34 (__client_streaming):
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\anaconda3\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)

In [16]:
import tkinter as tk
from PIL import Image, ImageTk
from vidstream import *
import socket
import threading
import requests

local_ip_address = socket.gethostbyname(socket.gethostname())
#print(local_ip_address)
server = StreamingServer(local_ip_address, 9493)
reciever = AudioReceiver(local_ip_address, 8483)

def start_listening():
    t1 = threading.Thread(target=server.start_server)
    t2 = threading.Thread(target=reciever.start_server)
    t1.start()
    t2.start()
    
    
def start_camera_stream():
    target_ip = target_ip_entry.get()
    camera_client = CameraClient(target_ip, 7473)
    t3 = threading.Thread(target=camera_client.start_stream)
    t3.start()

def start_screen_sharing():
    target_ip = target_ip_entry.get()
    screen_client = ScreenShareClient(target_ip, 7473)
    t4 = threading.Thread(target=screen_client.start_stream)
    t4.start()
    
def start_audio_stream():
    target_ip = target_ip_entry.get()
    audio_sender = AudioSender(target_ip, 6463)
    t5 = threading.Thread(target=audio_sender.start_stream)
    t5.start()



# Main window
root = tk.Tk()
root.title("Zoom Clone")

# Icon
#icon_image = Image.open("zoom.png")
#icon_photo = ImageTk.PhotoImage(icon_image)
#root.iconphoto(True, icon_photo)

# Icon
iconpath = r"C:\Users\KIIT\Desktop\5th sem materials\Project\zoom.ico"
root.iconbitmap(iconpath)

# Entry for Target IP
target_ip_label = tk.Label(root, text="Enter Target IP:")
target_ip_label.grid(row=0, column=0, padx=5, pady=5)
target_ip_entry = tk.Entry(root)
target_ip_entry.grid(row=0, column=1, padx=5, pady=5)

# Create a 2x2 grid for image buttons
button_width = 70
button_height = 70

listening_image = Image.open("play.png")
listening_image = listening_image.resize((button_width, button_height))
listening_photo = ImageTk.PhotoImage(listening_image)

camera_image = Image.open("camera.png")
camera_image = camera_image.resize((button_width, button_height))
camera_photo = ImageTk.PhotoImage(camera_image)

screen_share_image = Image.open("screen.png")
screen_share_image = screen_share_image.resize((button_width, button_height))
screen_share_photo = ImageTk.PhotoImage(screen_share_image)

audio_stream_image = Image.open("mic.png")
audio_stream_image = audio_stream_image.resize((button_width, button_height))
audio_stream_photo = ImageTk.PhotoImage(audio_stream_image)

listening_button = tk.Button(root, image=listening_photo, command=start_listening, width=button_width, height=button_height, compound=tk.TOP)
camera_stream_button = tk.Button(root, image=camera_photo, command=start_camera_stream, width=button_width, height=button_height, compound=tk.TOP)
screen_share_button = tk.Button(root, image=screen_share_photo, command=start_screen_sharing, width=button_width, height=button_height, compound=tk.TOP)
audio_stream_button = tk.Button(root, image=audio_stream_photo, command=start_audio_stream, width=button_width, height=button_height, compound=tk.TOP)

listening_button.grid(row=1, column=0, padx=5, pady=5)
camera_stream_button.grid(row=1, column=1, padx=5, pady=5)
screen_share_button.grid(row=2, column=0, padx=5, pady=5)
audio_stream_button.grid(row=2, column=1, padx=5, pady=5)

root.mainloop()

Exception in thread Thread-28 (__client_streaming):
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\anaconda3\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\KIIT\AppData\Roaming\Python\Python310\site-packages\vidstream\streaming.py", line 269, in __client_streaming
    self.__client_socket.connect((self.__host, self.__port))
TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond
Exception in thread Thread-30 (__client_streaming):
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\anaconda3\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)

In [10]:
import chardet

file_path = r"C:\Users\KIIT\Desktop\5th sem materials\Project\call_data.db"

with open(file_path, 'rb') as file:
    result = chardet.detect(file.read())

print(f"The detected encoding is: {result['encoding']}")


The detected encoding is: Windows-1252


In [9]:
import os

file_path = r'C:\sem materials\project\call_data.db'
os.chdir(os.path.dirname(file_path))  # Set the working directory to the directory containing the file



FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\sem materials\\project'

In [12]:
import codecs

# Specify the source and target file paths
source_file = r"C:\Users\KIIT\Desktop\5th sem materials\Project\call_data.db"
target_file =  r"C:\Users\KIIT\Desktop\5th sem materials\Projec\call_data_utf8.db"

# Read the source file with Windows-1252 encoding and write it to the target file in UTF-8 encoding
with codecs.open(source_file, 'r', encoding='Windows-1252') as source:
    with codecs.open(target_file, 'w', encoding='utf-8') as target:
        for line in source:
            target.write(line)


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\KIIT\\Desktop\\5th sem materials\\Projec\\call_data_utf8.db'

In [13]:
import codecs

# Specify the source and target file paths
source_file = r"C:\Users\KIIT\Desktop\5th sem materials\Project\call_data.db"
target_file = r"C:\Users\KIIT\Desktop\5th sem materials\Project\call_data_utf8.db"  # Corrected path

# Read the source file with Windows-1252 encoding and write it to the target file in UTF-8 encoding
with codecs.open(source_file, 'r', encoding='Windows-1252') as source:
    with codecs.open(target_file, 'w', encoding='utf-8') as target:
        for line in source:
            target.write(line)


UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 1529: character maps to <undefined>

In [15]:
import codecs

# Specify the source and target file paths
source_file = r"C:\Users\KIIT\Desktop\5th sem materials\Project\call_data.db"
target_file = r"C:\Users\KIIT\Desktop\5th sem materials\Project\call_data_utf8.db"  # Corrected path

# Read the source file with Windows-1252 encoding and write it to the target file in UTF-8 encoding
with codecs.open(source_file, 'r', encoding='Windows-1252', errors='ignore') as source:
    with codecs.open(target_file, 'w', encoding='utf-8') as target:
        for line in source:
            target.write(line)


In [1]:
import tkinter as tk
from PIL import Image, ImageTk
from vidstream import *
import socket
import threading
import requests

local_ip_address = socket.gethostbyname(socket.gethostname())
server = StreamingServer(local_ip_address, 1593)
reciever = AudioReceiver(local_ip_address, 1583)

def start_listening():
    try:
        t1 = threading.Thread(target=server.start_server)
        t2 = threading.Thread(target=reciever.start_server)
        t1.start()
        t2.start()
    except Exception as e:
        print(f"Error starting listening threads: {e}")

def start_camera_stream():
    target_ip = target_ip_entry.get()
    camera_client = CameraClient(target_ip, 1162)
    try:
        t3 = threading.Thread(target=camera_client.start_stream)
        t3.start()
    except Exception as e:
        print(f"Error starting camera streaming: {e}")

def start_screen_sharing():
    target_ip = target_ip_entry.get()
    screen_client = ScreenShareClient(target_ip, 1162)
    try:
        t4 = threading.Thread(target=screen_client.start_stream)
        t4.start()
    except Exception as e:
        print(f"Error starting screen sharing: {e}")

def start_audio_stream():
    target_ip = target_ip_entry.get()
    audio_sender = AudioSender(target_ip, 1172)
    try:
        t5 = threading.Thread(target=audio_sender.start_stream)
        t5.start()
    except Exception as e:
        print(f"Error starting audio streaming: {e}")

# Main window
root = tk.Tk()
root.title("Zoom Clone")

# Icon
iconpath = r"C:\Users\KIIT\Desktop\5th sem materials\Project\zoom.ico"
root.iconbitmap(iconpath)

# Entry for Target IP
target_ip_label = tk.Label(root, text="Enter Target IP:")
target_ip_label.grid(row=0, column=0, padx=5, pady=5)
target_ip_entry = tk.Entry(root)
target_ip_entry.grid(row=0, column=1, padx=5, pady=5)

# Create a 2x2 grid for image buttons
button_width = 70
button_height = 70

listening_image = Image.open("play.png")
listening_image = listening_image.resize((button_width, button_height))
listening_photo = ImageTk.PhotoImage(listening_image)

camera_image = Image.open("camera.png")
camera_image = camera_image.resize((button_width, button_height))
camera_photo = ImageTk.PhotoImage(camera_image)

screen_share_image = Image.open("screen.png")
screen_share_image = screen_share_image.resize((button_width, button_height))
screen_share_photo = ImageTk.PhotoImage(screen_share_image)

audio_stream_image = Image.open("mic.png")
audio_stream_image = audio_stream_image.resize((button_width, button_height))
audio_stream_photo = ImageTk.PhotoImage(audio_stream_image)

listening_button = tk.Button(root, image=listening_photo, command=start_listening, width=button_width, height=button_height, compound=tk.TOP)
camera_stream_button = tk.Button(root, image=camera_photo, command=start_camera_stream, width=button_width, height=button_height, compound=tk.TOP)
screen_share_button = tk.Button(root, image=screen_share_photo, command=start_screen_sharing, width=button_width, height=button_height, compound=tk.TOP)
audio_stream_button = tk.Button(root, image=audio_stream_photo, command=start_audio_stream, width=button_width, height=button_height, compound=tk.TOP)

listening_button.grid(row=1, column=0, padx=5, pady=5)
camera_stream_button.grid(row=1, column=1, padx=5, pady=5)
screen_share_button.grid(row=2, column=0, padx=5, pady=5)
audio_stream_button.grid(row=2, column=1, padx=5, pady=5)

root.mainloop()

In [1]:
import tkinter as tk
from vidstream import *
import socket
import threading
import time
import csv

# Specify the CSV file path
csv_file = "call_history.csv"

# Initialize dictionary to store call start times
call_start_times = {}

local_ip_address = socket.gethostbyname(socket.gethostname())
server = StreamingServer(local_ip_address, 6516)
reciever = AudioReceiver(local_ip_address, 6517)

def start_listening():
    t1 = threading.Thread(target=server.start_server)
    t2 = threading.Thread(target=reciever.start_server)
    t1.start()
    t2.start()
    # Record the start time of the call
    call_start_times[local_ip_address] = time.time()

def end_call(ip):
    global csv_file  # Add this line to access the global variable
    if ip in call_start_times:
        start_time = call_start_times[ip]
        end_time = time.time()
        call_duration = int(end_time - start_time)

        # Store the call data in the CSV file
        with open(csv_file, mode='a', newline='') as csv_file:
            fieldnames = ['user_ip', 'call_start_time', 'call_end_time', 'call_duration']
            writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

            writer.writerow({'user_ip': ip, 'call_start_time': start_time, 'call_end_time': end_time, 'call_duration': call_duration})

        print(f"Call with {ip} lasted for {call_duration} seconds")
    else:
        print(f"No call started with {ip}.")

camera_client = None  # Global variable to store the camera client

camera_client = None  # Global variable to store the camera client
camera_stream_active = False  # Flag to track if camera stream is active

def start_camera_stream():
    global camera_client, camera_stream_active
    
    target_ip = target_ip_entry.get()
    
    if camera_stream_active:
        # If camera stream is active, stop it
        camera_client.stop_stream()
        camera_stream_active = False
        camera_client = None
    else:
        # Start camera stream
        camera_client = CameraClient(target_ip, 6195)
        t3 = threading.Thread(target=camera_client.start_stream)
        t3.start()
        camera_stream_active = True



screen_client = None  # Global variable to store the screen sharing client
screen_share_active = False  # Flag to track if screen sharing is active

def start_screen_sharing():
    global screen_client, screen_share_active
    
    target_ip = target_ip_entry.get()
    
    if screen_share_active:
        # If screen sharing is active, stop it
        screen_client.stop_stream()
        screen_share_active = False
        screen_client = None
    else:
        # Start screen sharing
        screen_client = ScreenShareClient(target_ip, 6195)
        t4 = threading.Thread(target=screen_client.start_stream)
        t4.start()
        screen_share_active = True

audio_sender = None  # Global variable to store the audio sender
audio_stream_active = False  # Flag to track if audio stream is active

def start_audio_stream():
    global audio_sender, audio_stream_active
    
    target_ip = target_ip_entry.get()
    
    if audio_stream_active:
        # If audio stream is active, stop it
        audio_sender.stop_stream()
        audio_stream_active = False
        audio_sender = None
    else:
        # Start audio stream
        audio_sender = AudioSender(target_ip, 6196)
        t5 = threading.Thread(target=audio_sender.start_stream)
        t5.start()
        audio_stream_active = True


# Main window
root = tk.Tk()
root.title("Zoom Clone")

# Entry for Target IP
target_ip_label = tk.Label(root, text="Enter Target IP:")
target_ip_label.grid(row=0, column=0, padx=5, pady=5)
target_ip_entry = tk.Entry(root)
target_ip_entry.grid(row=0, column=1, padx=5, pady=5)

# Create buttons with text
button_width = 15
button_height = 2

listening_button = tk.Button(root, text="Start Listening", command=start_listening, width=button_width, height=button_height)
end_call_button = tk.Button(root, text="End Call", command=lambda: end_call(target_ip_entry.get()), width=button_width, height=button_height)
camera_stream_button = tk.Button(root, text="Start Camera Stream", command=start_camera_stream, width=button_width, height=button_height)
screen_share_button = tk.Button(root, text="Start Screen Sharing", command=start_screen_sharing, width=button_width, height=button_height)
audio_stream_button = tk.Button(root, text="Start Audio Stream", command=start_audio_stream, width=button_width, height=button_height)

listening_button.grid(row=1, column=0, padx=5, pady=5)
end_call_button.grid(row=1, column=1, padx=5, pady=5)
camera_stream_button.grid(row=2, column=0, padx=5, pady=5)
screen_share_button.grid(row=2, column=1, padx=5, pady=5)
audio_stream_button.grid(row=3, column=0, padx=5, pady=5)

root.mainloop()

Exception in thread Thread-6 (__client_streaming):
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\anaconda3\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\KIIT\AppData\Roaming\Python\Python310\site-packages\vidstream\audio.py", line 52, in __client_streaming
    self.__sending_socket.connect((self.__host, self.__port))
socket.gaierror: [Errno 11001] getaddrinfo failed
Exception in thread Thread-8 (__client_streaming):
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\anaconda3\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\KIIT\AppData\Roaming\Python\Python310\site-packages\vidstream\streaming.py", line 269, in __client_streaming
    self.__client_socket.connect((self.__ho

In [4]:
import tkinter as tk
from vidstream import *
import socket
import threading
import time
import csv

# Specify the CSV file path
csv_file = "call_history.csv"

# Initialize dictionary to store call start times
call_start_times = {}

local_ip_address = socket.gethostbyname(socket.gethostname())
server = StreamingServer(local_ip_address, 1518)
reciever = AudioReceiver(local_ip_address, 1519)

def start_listening():
    t1 = threading.Thread(target=server.start_server)
    t2 = threading.Thread(target=reciever.start_server)
    t1.start()
    t2.start()
    # Record the start time of the call
    call_start_times[local_ip_address] = time.time()

def end_call(ip):
    global csv_file  # Add this line to access the global variable
    if ip in call_start_times:
        start_time = call_start_times[ip]
        end_time = time.time()
        call_duration = int(end_time - start_time)

        # Store the call data in the CSV file
        with open(csv_file, mode='a', newline='') as csv_file:
            fieldnames = ['user_ip', 'call_start_time', 'call_end_time', 'call_duration']
            writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

            writer.writerow({'user_ip': ip, 'call_start_time': start_time, 'call_end_time': end_time, 'call_duration': call_duration})

        print(f"Call with {ip} lasted for {call_duration} seconds")
    else:
        print(f"No call started with {ip}.")

camera_client = None  # Global variable to store the camera client

camera_client = None  # Global variable to store the camera client
camera_stream_active = False  # Flag to track if camera stream is active

def start_camera_stream():
    global camera_client, camera_stream_active
    
    target_ip = target_ip_entry.get()
    
    if camera_stream_active:
        # If camera stream is active, stop it
        camera_client.stop_stream()
        camera_stream_active = False
        camera_client = None
    else:
        # Start camera stream
        camera_client = CameraClient(target_ip, 1197)
        t3 = threading.Thread(target=camera_client.start_stream)
        t3.start()
        camera_stream_active = True



screen_client = None  # Global variable to store the screen sharing client
screen_share_active = False  # Flag to track if screen sharing is active

def start_screen_sharing():
    global screen_client, screen_share_active
    
    target_ip = target_ip_entry.get()
    
    if screen_share_active:
        # If screen sharing is active, stop it
        screen_client.stop_stream()
        screen_share_active = False
        screen_client = None
    else:
        # Start screen sharing
        screen_client = ScreenShareClient(target_ip, 1197)
        t4 = threading.Thread(target=screen_client.start_stream)
        t4.start()
        screen_share_active = True

audio_sender = None  # Global variable to store the audio sender
audio_stream_active = False  # Flag to track if audio stream is active

def start_audio_stream():
    global audio_sender, audio_stream_active
    
    target_ip = target_ip_entry.get()
    
    if audio_stream_active:
        # If audio stream is active, stop it
        audio_sender.stop_stream()
        audio_stream_active = False
        audio_sender = None
    else:
        # Start audio stream
        audio_sender = AudioSender(target_ip, 1198)
        t5 = threading.Thread(target=audio_sender.start_stream)
        t5.start()
        audio_stream_active = True


# Main window
root = tk.Tk()
root.title("Zoom Clone")

# Entry for Target IP
target_ip_label = tk.Label(root, text="Enter Target IP:")
target_ip_label.grid(row=0, column=0, padx=5, pady=5)
target_ip_entry = tk.Entry(root)
target_ip_entry.grid(row=0, column=1, padx=5, pady=5)

# Create buttons with text
button_width = 15
button_height = 2

listening_button = tk.Button(root, text="Start Listening", command=start_listening, width=button_width, height=button_height)
end_call_button = tk.Button(root, text="End Call", command=lambda: end_call(target_ip_entry.get()), width=button_width, height=button_height)
camera_stream_button = tk.Button(root, text="Start Camera Stream", command=start_camera_stream, width=button_width, height=button_height)
screen_share_button = tk.Button(root, text="Start Screen Sharing", command=start_screen_sharing, width=button_width, height=button_height)
audio_stream_button = tk.Button(root, text="Start Audio Stream", command=start_audio_stream, width=button_width, height=button_height)

listening_button.grid(row=1, column=0, padx=5, pady=5)
end_call_button.grid(row=1, column=1, padx=5, pady=5)
camera_stream_button.grid(row=2, column=0, padx=5, pady=5)
screen_share_button.grid(row=2, column=1, padx=5, pady=5)
audio_stream_button.grid(row=3, column=0, padx=5, pady=5)

root.mainloop()